## 데이터 전처리

이 데이터로 모델을 학습시키기 위해선 각종 전처리가 필요합니다.

전처리에 사용할 함수들을 import하고, 각종 parameter를 설정합니다.

그 후, 해당 parameter들을 바탕으로 전처리를 진행합니다.

데이터 전처리가 어떻게 진행되는지 확인하기 위해 0번 데이터와 2번 데이터의 변화사항을 출력합니다.

### Tokenizing
1. Tokenizer instantiate
2. fit_on_texts
3. work_index : word index를 갖는 dictionary 생성
4. texts_to_sequences : 각 문장의 벡터값들을 하나의 sequence로 합치기
5. pad_sequences : 문장마다 길이가 다르므로 길이를 맞춰주기 위해서 0을 채움.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

vocab_size = 10000 # 단어사전의 크기
embedding_dim = 16 # 토큰 임베딩 차원 수
max_length = 120 # 입력 시퀀스의 최대 길이를 지정
trunc_type='post' # max_length보다 긴 시퀀스의 경우 어디를 잘라낼 것인지 지정. pre는 앞쪽을, post는 뒷쪽을 잘라냄
padding_type='post' # max_length보다 긴 시퀀스의 경우 어디에 padding을 붙일 것인지 지정. pre는 앞쪽에, post는 뒷쪽에 붙임
oov_tok = "<OOV>" # Out Of Vocabulary 토큰
training_size = 20000 # trainings_size만 train 데이터로 사용, 나머지는 test 데이터로 사용

print("원본 데이터 0번:", train_sentences[0])
print("원본 데이터 2번:", train_sentences[2])
print("#" * 30)

# 크기가 vocab_size인 단어사전을 만듦
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)

# 위에서 만든 단어사전을 바탕으로 단어들을 정수 인덱스로 변환
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)

print("정수로 인코딩 후 0번:", train_sequences[0])
print("길이:", len(train_sequences[0]))
print("정수로 인코딩 후 2번:", train_sequences[2])
print("길이:", len(train_sequences[2]))
print("#" * 30)

# pad_sequences 함수를 통해 padding 혹은 truncating하여 모든 데이터의 길이를 max_length로 통일
# 길이가 짧은 데이터는 padding을 추가하고, 길이가 긴 데이터는 trunc_type 설정에 따라 잘라냄

#  padding=padding_type 으로 옵션 주기도 한다.

train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type)
test_padded = pad_sequences(test_sequences, maxlen=max_length, truncating=trunc_type)

print("Padding/Truncating 후 0번:", train_padded[0])
print("길이:", len(train_padded[0]))
print("Padding/Truncating 후 2번:", train_padded[2])
print("길이:", len(train_padded[2]))

## Sarcasm 데이터셋을 사용해서 LSTM, Conv, GRU 모델 정의하기
## 네트워크 정의

tf.keras.models.Sequential을 이용해 RNN을 정의합니다.

먼저, 임베딩 레이어가 필요합니다.

임베딩 레이어는 단어사전 크기, 임베딩 크기, 시퀀스 길이를 argument로 받습니다.

<br>

임베딩 후에 RNN Layer를 추가합니다.

RNN Layer는 SimpleRNN, LSTM, GRU 등이 있습니다.

<br>

그 후 Dense Layer를 추가합니다.

일반적으로 RNN Layer 이후에 Dense Layer 하나를 먼저 추가하고,

task에 맞는 node 개수를 가진 Output Dense Layer를 추가합니다.

이진 분류 문제이므로 Output Layer에서는 sigmoid를 사용합니다.

<br>

이진 분류 문제이므로 binary_crossentropy를 loss로 사용하여 모델을 컴파일합니다.

`tf.keras.layers.Conv1D(128, 5, activation='relu`)
* Words will be **grouped** into the size of the filter in this case 5.
* We have 128 filters each 5 words.

In [ ]:
import numpy as np

import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sarcasm.json \
    -O /tmp/sarcasm.json

vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)


sentences = []
labels = []
urls = []
for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

# 크기가 vocab_size인 단어사전을 만듦
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index
# 위에서 만든 단어사전을 바탕으로 단어들을 정수 인덱스로 변환
training_sequences = tokenizer.texts_to_sequences(training_sentences)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)

# pad_sequences 함수를 통해 padding 혹은 truncating하여 모든 데이터의 길이를 max_length로 통일
# 길이가 짧은 데이터는 padding을 추가하고, 길이가 긴 데이터는 trunc_type 설정에 따라 잘라냄
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Convolution model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# GRU model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# LSTM, GRU 같이
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 50

training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=1)


In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
model.save("test.h5")

## 라벨분류하는 법 추가

In [ ]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

train_data, test_data = imdb['train'], imdb['test']

train_sentences = []
train_labels = []

test_sentences = []
test_labels = []

# 텍스트는 텍스트끼리, label은 label끼리 따로 분류
for s, l in train_data:

  # training_sentences.append(s.numpy().decode('utf8'))
  
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())
  
for s, l in test_data:
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())
  
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Stopwords 제공되면

In [ ]:
sentences = []
labels = []
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
print(len(stopwords))
# Expected Output
# 153

with open("/tmp/bbc-text.csv", 'r') as csvfile:
    # YOUR CODE HERE
    file = csv.reader(csvfile, delimiter=',')
    next(file) # remove header

    for row in file:
      labels.append(row[0])
      sentence = row[1]

      # remove stopwords
      for i in stopwords:
        token = " " + i + " " # 문장에서 불용어 리스트 중 하나라도 있으면
        removed_sentence = sentence.replace(token, " ") # 해당 불용어를 띄어쓰기로 대체(=불용어 삭제)
      sentences.append(removed_sentence)
    
print(len(labels))
print(len(sentences))
print(sentences[0])
# Expected Output
# 2225
# 2225
# tv future hands viewers home theatre systems  plasma high-definition tvs  digital video recorders moving living room  way people watch tv will radically different five years  time.  according expert panel gathered annual consumer electronics show las vegas discuss new technologies will impact one favourite pastimes. us leading trend  programmes content will delivered viewers via home networks  cable  satellite  telecoms companies  broadband service providers front rooms portable devices.  one talked-about technologies ces digital personal video recorders (dvr pvr). set-top boxes  like us s tivo uk s sky+ system  allow people record  store  play  pause forward wind tv programmes want.  essentially  technology allows much personalised tv. also built-in high-definition tv sets  big business japan us  slower take off europe lack high-definition programming. not can people forward wind adverts  can also forget abiding network channel schedules  putting together a-la-carte entertainment. us networks cable satellite companies worried means terms advertising revenues well  brand identity  viewer loyalty channels. although us leads technology moment  also concern raised europe  particularly growing uptake services like sky+.  happens today  will see nine months years  time uk   adam hume  bbc broadcast s futurologist told bbc news website. likes bbc  no issues lost advertising revenue yet. pressing issue moment commercial uk broadcasters  brand loyalty important everyone.  will talking content brands rather network brands   said tim hanlon  brand communications firm starcom mediavest.  reality broadband connections  anybody can producer content.  added:  challenge now hard promote programme much choice.   means  said stacey jolna  senior vice president tv guide tv group  way people find content want watch simplified tv viewers. means networks  us terms  channels take leaf google s book search engine future  instead scheduler help people find want watch. kind channel model might work younger ipod generation used taking control gadgets play them. might not suit everyone  panel recognised. older generations comfortable familiar schedules channel brands know getting. perhaps not want much choice put hands  mr hanlon suggested.  end  kids just diapers pushing buttons already - everything possible available   said mr hanlon.  ultimately  consumer will tell market want.   50 000 new gadgets technologies showcased ces  many enhancing tv-watching experience. high-definition tv sets everywhere many new models lcd (liquid crystal display) tvs launched dvr capability built  instead external boxes. one example launched show humax s 26-inch lcd tv 80-hour tivo dvr dvd recorder. one us s biggest satellite tv companies  directtv  even launched branded dvr show 100-hours recording capability  instant replay  search function. set can pause rewind tv 90 hours. microsoft chief bill gates announced pre-show keynote speech partnership tivo  called tivotogo  means people can play recorded programmes windows pcs mobile devices. reflect increasing trend freeing multimedia people can watch want  want.

# set train size using portion
train_size = int(len(sentences) * training_portion)

# 사이즈만큼 split
train_sentences = sentences[:train_size]
train_labels = labels[:train_size]

validation_sentences = sentences[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_sentences))
print(len(train_labels))
print(len(validation_sentences))
print(len(validation_labels))

# Expected output (if training_portion=.8)
# 1780
# 1780
# 1780
# 445
# 445